# Additional Tools and Patterns Roundup

### Integrating SQL

__SQL-based queries__ for initial couarse-grained feature extraction from datalakes/warehouses is a common pattern prior to more detailed feature engineering, modeling, etc.

If we have GPUs and our SQL isn't that complex, we can use BlazingSQL directly over large datasets in formats like parquet.

But what if we don't have GPUs available, or we have seriously gnarly SQL queries, perhaps inherited from earlier projects or architectures?

There's no magic bullet for this, but a few design decisions to consider:

1. Do we want to use a query engine that exposes a JDBC/ODBC interface, and access it from Python via Dask `read_sql_table`?
2. Or do we want to plan to run specific queries directly on the SQL engine and save the result set in a location & format that is convenient to read in with Python/Dask/etc.?
3. Spark or PrestoDB for our SQL engine? Are there legacy reasons that require Hive?
4. Do we need indices? If so, are we relying on existing Hive indices? Or can we use something like Apache Pinot or Intel OAP?

A common pattern is something like

#### <span style='color:green'>Data Lake $\leftarrow$ (Hive Metastore as Catalog) $\rightarrow$ SparkSQL $\rightarrow$ Parquet $\rightarrow$ HDFS or CN Storage $\rightarrow$ Dask</span>

### Logically, Sure ... But Practically I Need to Get High Perf Python Into My SparkSQL Query

In many cases, although we could extract all of the relevant data via SparkSQL and then operate on it later in Python, that may not be optimal.

For example, we may ...

* have a "table" that is 1000 columns wide, and we only needs 75 of those columns for the next analytics step ... but *which* 75 columns depends on the specific record
* have a feature store that defines 50 key features via math/normalization/imputation/etc. over subsets of those 1000 columns, and so we would like to run that feature definition logic in the query (so we get 50 meaningful feature columns only)
* want to model a large number of subsets of the raw rows, and query not the original data but a set of estimator/params for each group (e.g., a set of params describing transactions in each world postcode, not the raw transaction data)

In these cases, we can still use "real" high performance Python tools within SparkSQL via Spark's Vectorized PandasUDF facility.

https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html

Vectorized PandasUDF leverages batch ("vectorized") communication via Apache Arrow to move data between SparkSQL's internal byte-oriented data representation and Pandas/NumPy, which opens the door to much of SciPy, from NumPy to GPU-acceleration via RAPIDS, all within a normal SparkSQL query.

### What About Consuming Dense/Row Data?

If we are using deep learning tools to consume this data, there is an "impedance mismatch" between column-oriented formats like Parquet or ORC, and row-based data batches.

Uber's __Petastorm__ project is designed to overcome this mismatch in a way that maximizes throughput.

https://github.com/uber/petastorm

Petastorm includes APIs for pulling data to Tensorflow, PyTorch, or plain Python code, and has some nice extra features like a "metaschema" that allows NDArray/Tensor types with Parquet.

### Have You Seen the Size of Tensorflow & TFX Lately? I Just Need Tensors and Autograd

If you are working purely on deep learning, and your company has make a commitment to the full Tensorflow/TFX approach, then you'll likely try to arrange best practices within that large, growing, and changing ecosystem.

But for many data science groups, TF is just one of many tools, and TFX (along with constant changes to TF patterns and best practices) is too complex and expensive.

Even within Google, there are many groups that need accelerated tensor operations, autograd, and hardware acceleration ... and not much else.

What is a lot like PyTorch, but from Google instead?

<img src='images/jax.png'>

__"JAX: Autograd and XLA"__
* https://github.com/google/jax
* https://jax.readthedocs.io/en/latest/ 

### Yet Another Zero-Copy NumPy Impl with Accelerator Support

The headline pretty much covers it: __Bohrium__ 

https://bohrium.readthedocs.io/

JIT, operator fusion, and even accelerator support for AMD, Intel graphics, etc. via PyOpenCL

### Ray from Cal's RISE Lab

Optimized for safely maintaining state in massively distributed actor-based systems, Ray is a multi-language, fairly complex platform with Python bindings (among others). 

Ray is not yet widely used, but target applications for Ray include reinforcement learning and very-large-scale, stateful agent/cell-based modeling.

### So Many Options, How Can We Compare?

There are some folks working on HP-Python benchmarking, particularly https://github.com/dionhaefner/pyhpc-benchmarks